<a href="https://colab.research.google.com/github/francesco-dorati/DeepRL/blob/main/self_exercises/ppo/ppo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gymnasium[classic_control]
!pip install stable-baselines3[extra]
!pip install tensorboard
!pip install swig
!pip install "gymnasium[box2d]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 28.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp312-cp312-linux_x86_64.whl size=2409498 sha256=faac3ced9674eb5e578b0d28288cec50d55a8065570c6b9721866007aa484eab
  Stored in directory: /root/.cache/pip/wheels/2a/e9/60/774da0bcd07f7dc7761a8590fa2d065e4069568e78dcdc3318
Successfully built box2d-py


In [ ]:
import os
import time
import gymnasium as gym
import torch
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecVideoRecorder
from stable_baselines3.common.callbacks import BaseCallback
from torch.utils.tensorboard import SummaryWriter

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
# -----------------------------
# Parameters
# -----------------------------
ENV_ID = "CarRacing-v3"          # Gymnasium environment
TOTAL_TIMESTEPS = 1_000_000        # Training steps
NUM_ENVS = 1                    # Vectorized environments
VIDEO_FREQ = 100_000               # Record video every n steps
PPO_POLICY = "CnnPolicy"
N_STEPS = 4096
LEARNING_RATE = 5e-4
VIDEO_FOLDER = "videos"
RUN_NAME = f"{ENV_ID}_ppo_{int(time.time())}"
SEED = 42
VIDEO_LENGTH=1000
LOG_DIR = f"runs/{RUN_NAME}"

In [ ]:
os.makedirs(VIDEO_FOLDER, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

In [ ]:
# -----------------------------
# TensorBoard writer
# -----------------------------
writer = SummaryWriter(LOG_DIR)

In [ ]:
# -----------------------------
# Custom callback for logging
# -----------------------------
class TensorboardCallback(BaseCallback):
    def __init__(self, verbose=0):
        super().__init__(verbose)

    def _on_step(self) -> bool:
        if "episode" in self.locals['infos'][0]:
            ep_info = self.locals['infos'][0]['episode']
            writer.add_scalar("charts/episode_reward", ep_info['r'], self.num_timesteps)
            writer.add_scalar("charts/episode_length", ep_info['l'], self.num_timesteps)
        return True

In [ ]:
# -----------------------------
# Environment
# -----------------------------
def make_env(seed=0):
    def _init():
        env = gym.make(ENV_ID, render_mode="rgb_array")
        env = gym.wrappers.RecordEpisodeStatistics(env)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env
    return _init

In [ ]:
 env = DummyVecEnv([make_env(SEED + i) for i in range(NUM_ENVS)])

<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type swigvarlink has no __module__ attribute
/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.12/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.12/dist-packages/

In [ ]:
# Wrap video recording around one environment
env = VecVideoRecorder(
    env,
    VIDEO_FOLDER,
    record_video_trigger=lambda step: step % VIDEO_FREQ == 0,
    video_length=VIDEO_LENGTH,
    name_prefix=RUN_NAME
)

In [ ]:
# -----------------------------
# Model
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PPO(PPO_POLICY, env, verbose=1, device=device, n_steps=N_STEPS, learning_rate=LEARNING_RATE)
callback = TensorboardCallback()

Using cuda device
Wrapping the env in a VecTransposeImage.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
# -----------------------------
# Train
# -----------------------------
model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=callback)

Saving video to /content/videos/CarRacing-v3_ppo_1760475845-step-0-to-step-1000.mp4


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"


Moviepy - Building video /content/videos/CarRacing-v3_ppo_1760475845-step-0-to-step-1000.mp4.
Moviepy - Writing video /content/videos/CarRacing-v3_ppo_1760475845-step-0-to-step-1000.mp4



/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Moviepy - Done !
Moviepy - video ready /content/videos/CarRacing-v3_ppo_1760475845-step-0-to-step-1000.mp4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -52.7    |
| time/              |          |
|    fps             | 63       |
|    iterations      | 1        |
|    time_elapsed    | 64       |
|    total_timesteps | 4096     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1e+03       |
|    ep_rew_mean          | -52.1       |
| time/                   |             |
|    fps                  | 71          |
|    iterations           | 2           |
|    time_elapsed         | 114         |
|    total_timesteps      | 8192        |
| train/                  |             |
|    approx_kl            | 0.007452865 |
|    clip_fraction        | 0.0885      |
|    clip_range           | 0.2         |
|    en

...

Moviepy - Done !
Moviepy - video ready /content/videos/CarRacing-v3_ppo_1760475845-step-600000-to-step-601000.mp4
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 968        |
|    ep_rew_mean          | 58.4       |
| time/                   |            |
|    fps                  | 81         |
|    iterations           | 147        |
|    time_elapsed         | 7370       |
|    total_timesteps      | 602112     |
| train/                  |            |
|    approx_kl            | 0.06400524 |
|    clip_fraction        | 0.413      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.37      |
|    explained_variance   | 0.941      |
|    learning_rate        | 0.0005     |
|    loss                 | 0.0865     |
|    n_updates            | 1460       |
|    policy_gradient_loss | 0.00997    |
|    std                  | 0.393      |
|    value_loss           | 0.15       |
-------------------------

Moviepy - Done !
Moviepy - video ready /content/videos/CarRacing-v3_ppo_1760475845-step-700000-to-step-701000.mp4
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1e+03       |
|    ep_rew_mean          | -89.8       |
| time/                   |             |
|    fps                  | 82          |
|    iterations           | 172         |
|    time_elapsed         | 8591        |
|    total_timesteps      | 704512      |
| train/                  |             |
|    approx_kl            | 0.044187695 |
|    clip_fraction        | 0.267       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.45       |
|    explained_variance   | 0.93        |
|    learning_rate        | 0.0005      |
|    loss                 | -0.0293     |
|    n_updates            | 1710        |
|    policy_gradient_loss | -0.00732    |
|    std                  | 0.401       |
|    value_loss           | 0.016       |
----

Moviepy - Done !
Moviepy - video ready /content/videos/CarRacing-v3_ppo_1760475845-step-800000-to-step-801000.mp4
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1e+03     |
|    ep_rew_mean          | -89.8     |
| time/                   |           |
|    fps                  | 82        |
|    iterations           | 196       |
|    time_elapsed         | 9750      |
|    total_timesteps      | 802816    |
| train/                  |           |
|    approx_kl            | 0.0518995 |
|    clip_fraction        | 0.295     |
|    clip_range           | 0.2       |
|    entropy_loss         | -1.01     |
|    explained_variance   | 0.936     |
|    learning_rate        | 0.0005    |
|    loss                 | -0.0143   |
|    n_updates            | 1950      |
|    policy_gradient_loss | -0.00342  |
|    std                  | 0.352     |
|    value_loss           | 0.0164    |
---------------------------------------
------

Moviepy - Done !
Moviepy - video ready /content/videos/CarRacing-v3_ppo_1760475845-step-900000-to-step-901000.mp4
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1e+03      |
|    ep_rew_mean          | -90.8      |
| time/                   |            |
|    fps                  | 82         |
|    iterations           | 220        |
|    time_elapsed         | 10919      |
|    total_timesteps      | 901120     |
| train/                  |            |
|    approx_kl            | 0.07501649 |
|    clip_fraction        | 0.321      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.636     |
|    explained_variance   | 0.968      |
|    learning_rate        | 0.0005     |
|    loss                 | -0.0356    |
|    n_updates            | 2190       |
|    policy_gradient_loss | 0.000643   |
|    std                  | 0.313      |
|    value_loss           | 0.01       |
-------------------------

Moviepy - Done !
Moviepy - video ready /content/videos/CarRacing-v3_ppo_1760475845-step-1000000-to-step-1001000.mp4
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1e+03       |
|    ep_rew_mean          | -90.6       |
| time/                   |             |
|    fps                  | 82          |
|    iterations           | 245         |
|    time_elapsed         | 12127       |
|    total_timesteps      | 1003520     |
| train/                  |             |
|    approx_kl            | 0.068631895 |
|    clip_fraction        | 0.316       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.222      |
|    explained_variance   | 0.834       |
|    learning_rate        | 0.0005      |
|    loss                 | -0.0204     |
|    n_updates            | 2440        |
|    policy_gradient_loss | -0.00875    |
|    std                  | 0.272       |
|    value_loss           | 0.0106      |
--

In [ ]:
# -----------------------------
# Save model
# -----------------------------
model.save(os.path.join(LOG_DIR, "ppo_model"))

In [ ]:
import numpy as np

# -----------------------------
# Evaluate PPO Model
# -----------------------------
n_eval_episodes = 5  # Number of episodes to evaluate
max_steps_per_episode = 200

episode_rewards = []  # Store total rewards for each episode

for episode in range(n_eval_episodes):
    obs = env.reset()
    done = [False]
    total_reward = 0

    for step in range(max_steps_per_episode):
        # Predict deterministic actions (no exploration)
        action, _ = model.predict(obs, deterministic=True)

        # Step the environment
        obs, reward, done, info = env.step(action)
        total_reward += reward[0]

        # Stop if the episode ends
        if done[0]:
            break

    episode_rewards.append(total_reward)
    print(f"Episode {episode + 1}/{n_eval_episodes} | Total Reward: {total_reward:.2f}")

# -----------------------------
# Print Summary Report
# -----------------------------
mean_reward = np.mean(episode_rewards)
std_reward = np.std(episode_rewards)
print("\n================ Evaluation Report ================")
print(f"Evaluated over {n_eval_episodes} episodes")
print(f"Average Reward: {mean_reward:.2f}")
print(f"Reward Std Dev: {std_reward:.2f}")
print("===================================================")

In [ ]:
env.close()
writer.close()
print(f"✅ Training complete!\nTensorBoard logs: {LOG_DIR}\nVideos: {VIDEO_FOLDER}")